#### TensorBoard
```python
from tensorflow.keras.callbacks import TensorBoard

# set log directory
log_directory = 'logs/fit'

# add a timestamp if needed
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H%M")
log_directory = log_directory + '/' + timestamp

# Set Board
board = TensorBoard(log_dir=log_directory,histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=1)

# While fitting the model, add to callbacks
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop,board])

# run this code at command line
# make sure you change directory to your notebook
tensorboard --logdir logs\fit 
```
Tensorboard will run locally in your browser at 
[http://localhost:6006/](http://localhost:6006/)


#### One Hot Encoding
```python
from tensorflow.keras.utils import to_categorical
y_encoded= to_categorical(y)
```

#### Save and Load a Model
```python
model.save('my_model.h5')  # creates a HDF5 file
from tensorflow.keras.models import load_model
later_model = load_model('my_model.h5')
```

# ANN

#### Regression


```python
# Normalizing/Scaling the Data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train) # we only fit our scaler to the training set
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Import Keras Model, Layers, ...
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error

# Build the model
model = Sequential()
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse') # we can use our imported optimizer or loss function

# Train the model
model.fit(x=X_train,y=y_train.values,
          validation_data=(X_test,y_test.values),
          batch_size=128,epochs=400)

# Losses
losses = pd.DataFrame(model.history.history)

# Predictions
y_hat = model.predict(X_test)
```

#### Classification


```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=15,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam') # For Binary Classification

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop])

model_loss = pd.DataFrame(model.history.history)
```

# CNN

#### Graysclae Images 2D

```python
# First Step
Preprocessing the data:
    One-Hot Encoding
    Scaling
    Reshaping
    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Build the model
model = Sequential()

# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(28, 28, 1), activation='relu',))
# POOLING LAYER
model.add(MaxPool2D(pool_size=(2, 2)))

# FLATTEN IMAGES FROM 28 by 28 to 764 BEFORE FINAL LAYER
model.add(Flatten())

# 128 NEURONS IN DENSE HIDDEN LAYER (YOU CAN CHANGE THIS NUMBER OF NEURONS)
model.add(Dense(128, activation='relu'))

# LAST LAYER IS THE CLASSIFIER, THUS 10 POSSIBLE CLASSES
model.add(Dense(10, activation='softmax'))

# https://keras.io/metrics/
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) # we can add in additional metrics https://keras.io/metrics/

# Early Stopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

# Train model
model.fit(x_train,y_cat_train,epochs=10,validation_data=(x_test,y_cat_test),callbacks=[early_stop])

# Evaluate model
losses = pd.DataFrame(model.history.history)
losses[['accuracy','val_accuracy']].plot()
losses[['loss','val_loss']].plot()

print(model.metrics_names)
print(model.evaluate(x_test,y_cat_test,verbose=0))

from sklearn.metrics import classification_report,confusion_matrix
predictions = model.predict_classes(x_test)
print(classification_report(y_test,predictions))
sns.heatmap(confusion_matrix(y_test,predictions),annot=True)

```

In [ ]:
to_ca